In [1]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [3]:
# Read in nasdaq data and extract symbols column
nasdaq_list = pd.read_csv("nasdaq_list.csv")
# Pull API key from .env file
FMP_API_KEY = config("FMP_API_KEY")

nasdaq_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8202 entries, 0 to 8201
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      8202 non-null   object 
 1   Name        8202 non-null   object 
 2   Market Cap  7729 non-null   float64
 3   Country     7458 non-null   object 
 4   IPO Year    4820 non-null   float64
 5   Sector      6178 non-null   object 
 6   Industry    6195 non-null   object 
dtypes: float64(2), object(5)
memory usage: 448.7+ KB
None


In [4]:
# Filter missing out rows with missing data
# Filter reduces dataset from 8202 entries to 3156 entries
nasdaq_list_filtered = nasdaq_list.dropna()
nasdaq_list_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3156 entries, 0 to 8200
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      3156 non-null   object 
 1   Name        3156 non-null   object 
 2   Market Cap  3156 non-null   float64
 3   Country     3156 non-null   object 
 4   IPO Year    3156 non-null   float64
 5   Sector      3156 non-null   object 
 6   Industry    3156 non-null   object 
dtypes: float64(2), object(5)
memory usage: 197.2+ KB
None


In [5]:
symbols = nasdaq_list_filtered.Symbol

url_list = []
for idx, val in enumerate(symbols):
    url = "https://financialmodelingprep.com/api/v3/earnings-surprises/{}?apikey={}".format(val, FMP_API_KEY)
    url_list.append(url)

In [6]:
# Call FMP API for each URL using Concurrent library
# Run takes 211 seconds ... be patient
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(get_jsonparsed_data, url) for url in url_list]
    concurrent.futures.wait(res)

In [12]:
df = pd.DataFrame()
for x in range(len(symbols)):
    res_df = pd.DataFrame(res[x].result())
    df = pd.concat([df, res_df])

In [13]:
print(df)

          date symbol  actualEarningResult  estimatedEarning
0   2021-11-22      A                 1.21              1.17
1   2021-08-17      A                 1.10              0.99
2   2021-05-25      A                 0.97              0.83
3   2021-02-16      A                 1.06              0.89
4   2020-11-23      A                 0.98              0.93
..         ...    ...                  ...               ...
8   2019-11-06   ZYNE                -0.08             -0.52
9   2019-08-06   ZYNE                -0.50             -0.50
10  2019-05-08   ZYNE                -0.47             -0.49
11  2019-03-11   ZYNE                -0.44             -0.65
12  2018-11-08   ZYNE                -0.47             -0.70

[21702 rows x 4 columns]


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21702 entries, 0 to 21701
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 21702 non-null  object 
 1   symbol               21702 non-null  object 
 2   actualEarningResult  21702 non-null  float64
 3   estimatedEarning     21701 non-null  float64
dtypes: float64(2), object(2)
memory usage: 678.3+ KB


In [20]:
df.to_csv("earnings_surprises.csv")